# Libs

In [1]:
import yfinance as yf

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 863.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing ins

In [5]:
!pip install torch

In [6]:
!pip install torch torch-geometric pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 704.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:00


# Inital pull of data to see what it looks like

In [ ]:
stocks = ["AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", "TSLA", "NVDA", "META", "UNH", "JNJ",
          "V", "JPM", "PG", "HD", "MA", "BAC", "XOM", "CVX", "LLY", "PFE"]

data = yf.download(stocks, period="1mo")
df = pd.DataFrame(data)
df.to_csv("stock_data.csv")

[*********************100%***********************]  20 of 20 completed


In [ ]:
df.head()

Price                       Adj Close                                     \
Ticker                           AAPL        AMZN        BAC         CVX   
Date                                                                       
2024-08-19 00:00:00+00:00  225.889999  178.220001  39.413040  146.830002   
2024-08-20 00:00:00+00:00  226.509995  178.880005  38.439388  144.690002   
2024-08-21 00:00:00+00:00  226.399994  180.110001  38.499001  145.320007   
2024-08-22 00:00:00+00:00  224.529999  176.130005  38.985828  145.729996   
2024-08-23 00:00:00+00:00  226.839996  177.039993  39.512398  147.619995   

Price                                                                      \
Ticker                           GOOG       GOOGL          HD         JNJ   
Date                                                                        
2024-08-19 00:00:00+00:00  168.178604  166.449127  360.878082  158.427521   
2024-08-20 00:00:00+00:00  168.737885  166.958450  365.052704  158.953522   
2024-08-21 00:00:00+00:00  167.409637  165.630219  368.223450  160.213943   
2024-08-22 00:00:00+00:00  165.272446  163.582932  363.154236  161.127029   
2024-08-23 00:00:00+00:00  167.209885  165.400513  373.322479  162.893616   

Price                                              ...   Volume            \
Ticker                            JPM         LLY  ...       MA      META   
Date                                               ...                      
2024-08-19 00:00:00+00:00  215.449997  921.809998  ...  1690800   9879700   
2024-08-20 00:00:00+00:00  214.520004  949.969971  ...  1763300   7944400   
2024-08-21 00:00:00+00:00  214.600006  951.969971  ...  2105900  13423300   
2024-08-22 00:00:00+00:00  216.630005  954.169983  ...  1538200  15708300   
2024-08-23 00:00:00+00:00  218.309998  952.739990  ...  1986300  11323900   

Price                                                                        \
Ticker                         MSFT       NVDA       PFE       PG      TSLA   
Date                                                                          
2024-08-19 00:00:00+00:00  15234000  318333600  26382500  3885300  76435200   
2024-08-20 00:00:00+00:00  16387600  300087400  21301300  4513600  74001200   
2024-08-21 00:00:00+00:00  16067300  257883600  16729700  4082000  70146000   
2024-08-22 00:00:00+00:00  19361900  376189100  20241500  4097300  79514500   
2024-08-23 00:00:00+00:00  18493800  323230300  21885300  4771300  81525200   

Price                                                  
Ticker                         UNH        V       XOM  
Date                                                   
2024-08-19 00:00:00+00:00  1847900  6555900  12417500  
2024-08-20 00:00:00+00:00  1677500  6803600  15632000  
2024-08-21 00:00:00+00:00  2844300  6639700  11752400  
2024-08-22 00:00:00+00:00  2282000  4425200  10609500  
2024-08-23 00:00:00+00:00  2376400  3697400  10381400  

[5 rows x 120 columns]

# Fine tuned data API call

just pulling the stock data would only give us the prices at the end of that, that is not what we would be looking at for our project, we would want some more data that we could base our model off.

In [7]:
info = ['sector', 'industry', 'fullTimeEmployees', 'profitMargins',
        'operatingMargins', 'returnOnAssets', 'returnOnEquity',
        'revenueGrowth', 'earningsGrowth', 'debtToEquity', 'totalCash',
        'totalDebt', 'totalRevenue', 'bookValue', 'operatingCashflow',
        'freeCashflow', 'targetLowPrice', 'targetMeanPrice',
        'targetMedianPrice', 'recommendationMean']

data = []
stocks = ["AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", "TSLA", "NVDA", "META", "UNH", "JNJ", "V", "JPM", "PG", "HD", "MA", "BAC", "XOM", "CVX", "LLY", "PFE"]

# Download closing prices
prices = yf.download(stocks, period="1d")['Close']

for stock in stocks:
  ticker = yf.Ticker(stock)
  stock_info = [stock]
  for i in info:
    try:
      stock_info.append(ticker.info[i])
    except KeyError:
      stock_info.append(None)
  # Add closing price to stock_info
  stock_info.append(prices[stock].iloc[0])
  data.append(stock_info)

df = pd.DataFrame(data, columns = ['Ticker'] + info + ['Close'])

[*********************100%***********************]  20 of 20 completed


In [8]:
df.head()

,Ticker,sector,industry,fullTimeEmployees,profitMargins,operatingMargins,returnOnAssets,returnOnEquity,revenueGrowth,earningsGrowth,...,totalDebt,totalRevenue,bookValue,operatingCashflow,freeCashflow,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationMean,Close
0,AAPL,Technology,Consumer Electronics,161000.0,0.26441,0.29556,0.22612,1.60583,0.049,0.111,...,101304000512,385603010560,4.382,113040998400,8.615812e+10,165.20,216.16,218.34,2.0,216.789993
1,MSFT,Technology,Software - Infrastructure,228000.0,0.35956,0.43143,0.14802,0.37133,0.152,0.097,...,97851998208,245122007040,36.115,118547996672,5.670525e+10,395.34,446.26,449.25,1.7,435.149994
2,AMZN,Consumer Cyclical,Internet Retail,1525000.0,0.07350,0.09915,0.06584,0.21933,0.101,0.938,...,157841997824,604333998080,22.540,107951996928,5.919100e+10,161.73,196.19,197.67,1.8,186.880005
3,GOOG,Communication Services,Internet Content & Information,179582.0,0.26702,0.32363,0.15961,0.30871,0.136,0.314,...,28718999552,328284012544,24.408,105059000320,4.398850e+10,130.28,180.22,183.75,1.8,160.279999
4,GOOGL,Communication Services,Internet Content & Information,179582.0,0.26702,0.32363,0.15961,0.30871,0.136,0.314,...,28718999552,328284012544,24.408,105059000320,4.398850e+10,170.00,201.46,200.00,1.9,159.320007


In [9]:
df['Close']

,Close
0,216.789993
1,435.149994
2,186.880005
3,160.279999
4,159.320007
5,227.869995
6,115.589996
7,536.315002
8,577.960022
9,167.070007


In [10]:
df.to_csv("stock_data.csv")

# Modeling

In [11]:
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [12]:
print(df.isna().sum())

Ticker                0
sector                0
industry              0
fullTimeEmployees     1
profitMargins         0
operatingMargins      0
returnOnAssets        0
returnOnEquity        0
revenueGrowth         0
earningsGrowth        0
debtToEquity          2
totalCash             0
totalDebt             0
totalRevenue          0
bookValue             0
operatingCashflow     0
freeCashflow          2
targetLowPrice        0
targetMeanPrice       0
targetMedianPrice     0
recommendationMean    0
Close                 0
dtype: int64


In [13]:
df_cleaned = df.dropna()

In [14]:
df_cleaned = df.dropna(how='any').reset_index(drop=True)

In [15]:
df_cleaned.isna().sum()

,0
Ticker,0
sector,0
industry,0
fullTimeEmployees,0
profitMargins,0
operatingMargins,0
returnOnAssets,0
returnOnEquity,0
revenueGrowth,0
earningsGrowth,0


In [16]:
from sklearn.preprocessing import StandardScaler


df_encoded = pd.get_dummies(df_cleaned.drop(columns=['Ticker']), columns=['sector', 'industry'], drop_first=True)
df_encoded = df_encoded.astype(int)

features = ['fullTimeEmployees', 'profitMargins', 'operatingMargins', 'returnOnAssets',
            'returnOnEquity', 'revenueGrowth', 'earningsGrowth', 'debtToEquity',
            'totalCash', 'totalDebt', 'totalRevenue', 'bookValue',
            'operatingCashflow', 'freeCashflow', 'targetLowPrice',
            'targetMeanPrice', 'targetMedianPrice', 'recommendationMean']

scaler = StandardScaler()
df_encoded[features] = scaler.fit_transform(df_encoded[features])



In [17]:
df_encoded.head()

,fullTimeEmployees,profitMargins,operatingMargins,returnOnAssets,returnOnEquity,revenueGrowth,earningsGrowth,debtToEquity,totalCash,totalDebt,...,industry_Credit Services,industry_Drug Manufacturers - General,industry_Healthcare Plans,industry_Home Improvement Retail,industry_Household & Personal Products,industry_Internet Content & Information,industry_Internet Retail,industry_Oil & Gas Integrated,industry_Semiconductors,industry_Software - Infrastructure
0,-0.202730,0.0,0.0,0.0,0.443398,-0.25,-0.25,0.005562,0.663202,1.316287,...,0,0,0,0,0,0,0,0,0,0
1,-0.009328,0.0,0.0,0.0,-0.394132,-0.25,-0.25,-0.334222,1.067419,1.225477,...,0,0,0,0,0,0,0,0,0,1
2,3.734593,0.0,0.0,0.0,-0.394132,-0.25,-0.25,-0.245583,1.466660,2.803591,...,0,0,0,0,0,0,1,0,0,0
3,-0.149092,0.0,0.0,0.0,-0.394132,-0.25,-0.25,-0.413997,1.809141,-0.593155,...,0,0,0,0,0,1,0,0,0,0
4,-0.149092,0.0,0.0,0.0,-0.394132,-0.25,-0.25,-0.413997,1.809141,-0.593155,...,0,0,0,0,0,1,0,0,0,0


In [18]:
X = df_encoded.drop(columns=['Close']).values
y = df_encoded['Close'].values

# Convert to torch tensors
x = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)

In [19]:
edge_index = []

for i in range(len(df_cleaned)):
    for j in range(i + 1, len(df_cleaned)):
        if df_cleaned.iloc[i]['sector'] == df.iloc[j]['sector']:
            edge_index.append([i, j])
            edge_index.append([j, i])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

data = Data(x=x, edge_index=edge_index, y=y)

In [20]:
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels=x.shape[1], out_channels=64)
        self.conv2 = GCNConv(in_channels=64, out_channels=32)
        self.fc = nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.fc(x)
        return x

model = GNN()

In [21]:
import torch.optim as optim


optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output.flatten(), data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 136131.09375
Epoch 10, Loss: 135940.125
Epoch 20, Loss: 135699.875
Epoch 30, Loss: 135365.40625
Epoch 40, Loss: 134858.640625
Epoch 50, Loss: 134081.234375
Epoch 60, Loss: 132926.40625
Epoch 70, Loss: 131218.296875
Epoch 80, Loss: 128784.0625
Epoch 90, Loss: 125515.03125
Epoch 100, Loss: 121303.3125
Epoch 110, Loss: 116069.3515625
Epoch 120, Loss: 109781.53125
Epoch 130, Loss: 102494.3046875
Epoch 140, Loss: 94345.015625
Epoch 150, Loss: 85576.28125
Epoch 160, Loss: 76496.03125
Epoch 170, Loss: 67527.546875
Epoch 180, Loss: 59109.859375
Epoch 190, Loss: 51644.48046875


While the results were not super accurate as of yet, there is alot of room to grow here and build off this. This is a simple config of this idea and there was not alot of fine tuning done yet, there is alot more information that can be used to bring those numbers much closer to each other. This is a good base of ideas from other projects along with my research of many different papers for our group to use moving forward and build off this idea


In [22]:
model.eval()
with torch.no_grad():
    predictions = model(data).flatten().detach().numpy()

actual_prices = y.detach().numpy()
tickers = df_cleaned['Ticker'].values

comparison_df = pd.DataFrame({'Ticker': tickers, 'Actual Price': actual_prices, 'Predicted Price': predictions})
print(comparison_df)

   Ticker  Actual Price  Predicted Price
0    AAPL         216.0       203.752914
1    MSFT         435.0       203.752914
2    AMZN         186.0       117.664093
3    GOOG         160.0       160.241638
4   GOOGL         159.0       160.241638
5    TSLA         227.0       117.664093
6    NVDA         115.0       203.752914
7    META         536.0       160.241638
8     UNH         577.0       243.329849
9     JNJ         167.0       243.329849
10     PG         175.0       241.971146
11     HD         383.0       133.219650
12     MA         500.0       370.964905
13    XOM         114.0       183.700653
14    CVX         143.0       225.790131
15    LLY         906.0       335.180786
16    PFE          29.0       149.528137


In [23]:
results_df = pd.DataFrame({'Ticker': df_cleaned['Ticker'], 'Predicted Price': predictions})
print(results_df)

   Ticker  Predicted Price
0    AAPL       203.752914
1    MSFT       203.752914
2    AMZN       117.664093
3    GOOG       160.241638
4   GOOGL       160.241638
5    TSLA       117.664093
6    NVDA       203.752914
7    META       160.241638
8     UNH       243.329849
9     JNJ       243.329849
10     PG       241.971146
11     HD       133.219650
12     MA       370.964905
13    XOM       183.700653
14    CVX       225.790131
15    LLY       335.180786
16    PFE       149.528137


In [24]:
import gradio as gr

In [25]:
def display_table():
  return results_df

In [26]:
interface = gr.Interface(
    fn=display_table,
    inputs=None,
    outputs="dataframe",
    title="Market Predictions (Not Financial Advice)"
)

In [27]:
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1eb59eb847da45b337.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Trouble shooting

In [ ]:
np.isinf(df_encoded).sum()

,0
fullTimeEmployees,0
profitMargins,0
operatingMargins,0
returnOnAssets,0
returnOnEquity,0
revenueGrowth,0
earningsGrowth,0
debtToEquity,0
totalCash,0
totalDebt,0


In [ ]:
print(torch.isnan(x).sum())
print(torch.isinf(x).sum())
print(torch.isnan(y).sum())
print(torch.isinf(y).sum())

tensor(0)
tensor(0)
tensor(0)
tensor(0)


In [ ]:
print(output)

tensor([[102.7459],
        [102.7459],
        [ 57.2953],
        [ 74.2439],
        [ 74.2439],
        [ 57.2953],
        [102.7459],
        [ 74.2439],
        [144.0536],
        [144.0536],
        [152.0555],
        [ 72.1495],
        [232.3983],
        [ 96.7878],
        [140.5052],
        [209.1741],
        [ 90.7337]], grad_fn=<AddmmBackward0>)
